In [1]:
import os
from datetime import date, timedelta
import glob
import time
import datetime
import pandas as pd
import numpy as np
import openpyxl
import win32com.client as win32

In [2]:
dt = datetime.datetime.now()
dt_string=dt.strftime("%#m.%#d.%Y")
print(dt_string)


2.16.2024


In [ ]:
path=r"D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\Korea RSL File to send email\Korea PO Detail RSL Expiry Date.xlsx"
by_brand_path=r"D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\Korea RSL File to send email\By Brand"
by_factory_path=r"D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\Korea RSL File to send email\By Factory Group"
# control_file=r"D:\User\nguyenta\OneDrive - Wolverine World Wide\Desktop\Crawl Data\PO Details - RSL EXPIRY - Korea - Control List.xlsx"
control_file=r"D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\RSL Control Folder\PO Details - RSL EXPIRY - Korea - Control List.xlsx"

In [ ]:
control=pd.read_excel(control_file)
(max_row,max_col)=control.shape
control.head()

In [ ]:
control_unpivot=pd.melt(control, 
                       id_vars = ['Brand','Factory Group'], 
                       value_vars = [i for i in control.columns if i.startswith('Email')], 
                       var_name = '#No', 
                       value_name = 'Email'
                    );
control_unpivot.dropna(subset=['Email'],inplace=True)
# control_unpivot = control_unpivot[control_unpivot['Brand','Factory Group','Email'].duplicated(keep=False)]
# control_unpivot['All_email']=control_unpivot.groupby(['Brand','Factory Group'])['Email'].agg(lambda x: ';'.join(set(x))).re
df = pd.DataFrame(
    control_unpivot.groupby(['Brand','Factory Group'])
      .apply(
          lambda control_unpivot: ';'.join(control_unpivot['Email'].unique())
      ),
columns=['All_email']).reset_index()
df

In [ ]:
df["All_email"]=df["All_email"].str.replace("\n",";")
df["Factory Group"]=df["Factory Group"].str.replace("/","-")
df

In [7]:
def refresh_data():
    xlapp=win32.Dispatch('Excel.Application')
    #Open workbook and refresh data
    xlapp.DisplayAlerts = False
    xlapp.Visible = True
    wb=xlapp.Workbooks.Open(path)
    wb.RefreshAll()
    time.sleep(10)
    xlapp.CalculateUntilAsyncQueriesDone()
    wb.Save()
    xlapp.Quit()

In [18]:
data=pd.read_excel(path,skiprows=2)
data.head()
data_copy=data.copy()
data_new=data_copy.rename(columns={'PO Details - RSL EXPIRY[Brand]':'Brand',
                                    'PO Details - RSL EXPIRY[Factory]':'Factory',
                                    'PO Details - RSL EXPIRY[PO#]':'PO#',
                                    'PO Details - RSL EXPIRY[Tracking#]':'Tracking#',
                                    'PO Details - RSL EXPIRY[STK#]':'STK#',
                                    'PO Details - RSL EXPIRY[Style]':'Style',
                                    'PO Details - RSL EXPIRY[X/F Month Year]':'X/F Month Year',
                                    'PO Details - RSL EXPIRY[Current XF Date]':'X/F Date',
                                    'PO Details - RSL EXPIRY[Vendor Number]':'Vendor Number',
                                    'All_Factory[Factory Group]':'Factory Group',
                                    'PO Details - RSL EXPIRY[Korea RSL EXPIRY DATE]':'Korea RSL EXPIRY DATE',
                                    'PO Details - RSL EXPIRY[Korea RSL Status]':'Korea RSL Status',
                                    '[Pairs]':'Pairs'
                                    }
                ) 
data_new.columns

Index(['Brand', 'Factory', 'PO#', 'Tracking#', 'STK#', 'Style',
       'X/F Month Year', 'X/F Date', 'Vendor Number', 'Factory Group',
       'Korea RSL EXPIRY DATE', 'Korea RSL Status', 'Pairs'],
      dtype='object')

In [19]:
data_new.shape

(621, 13)

In [20]:
brand=data_new["Brand"].unique()
brand

array(['Chaco', 'Harley', 'Hush Puppies', 'Keds', 'Merrell',
       'Merrell Work', 'Saucony', 'Sperry'], dtype=object)

In [21]:
data_new["Factory Group"]

0       D'CLASE
1       D'CLASE
2       D'CLASE
3       D'CLASE
4       D'CLASE
         ...   
616    HSIEH DA
617    HSIEH DA
618    HSIEH DA
619    HSIEH DA
620    HSIEH DA
Name: Factory Group, Length: 621, dtype: object

In [22]:
data_new["Factory Group"]=data_new["Factory Group"].str.replace("/","-")
f_group=data_new["Factory Group"].unique()
f_group

array(["D'CLASE", 'SCS', 'SPORTS WORLD', 'FENGDUN-MIRAMA', 'BEST HEALTH',
       'XDDD', 'SHENGSHENG', 'FIRST STEP', 'STELLA', 'A-GLORY',
       'CHINOOK AGENT', 'FULGENT SUN', 'KING MAKER', 'TBS', 'POU CHEN',
       'AMG AGENT', 'XIECHENG', 'XINXIESHENG', 'RIGHT RICH', 'SAMDUK',
       'JIATAI', "BITI'S", 'GREEN LAND', 'SEMASI', 'HSIEH DA',
       'YANGZHOU DIYE-AGT', 'XUSHENG'], dtype=object)

In [76]:
def delete_oldfile(to_date: str):
    for folder_path in os.listdir(by_brand_path):
        if folder_path.endswith('.xlsx'):
            del_path=f"{by_brand_path}\\{folder_path}"
            modification_time=os.path.getmtime(del_path)
            os.remove(del_path)
            time.sleep(2)
            print(f"{datetime.datetime.now()}-[get_report] delete {del_path}")
# if __name__ == "__main__":
#     delete_oldfile(to_date=datetime.datetime.now().strftime("%d.%m.%Y %H%M"))

In [221]:
#Split file by Brand:
for folder_path in os.listdir(by_brand_path):
        if folder_path.endswith('.xlsx'):
            del_path=f"{by_brand_path}\\{folder_path}"
            modification_time=os.path.getmtime(del_path)
            os.remove(del_path)
#             time.sleep(2)
            print(f"{datetime.datetime.now()}-[get_report] delete {del_path}")
for b in brand:
    df1=data_new[data_new["Brand"]==b]
    df1.to_excel(f"{by_brand_path}\\{b} Korea PO Detail-RSL Expiry.xlsx", index=False)
for file in os.listdir(by_brand_path):
    if file.endswith('.xlsx'):
        filepath=f"{by_brand_path}\\{file}"
        df = pd.read_excel(filepath)
        (max_row,max_col)=df.shape
        print(df.shape)
        with pd.ExcelWriter(filepath, engine='xlsxwriter') as writer:
            df.to_excel(writer, sheet_name="Sheet1",startrow=0,index=False)
            auto_adjust_xlsx_column_width(df, writer, sheet_name="Sheet1",margin=0)
#             column_settings = [{'header': column} for column in df.columns]
            workbook  = writer.book
            worksheet = writer.sheets['Sheet1']
#             worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
            worksheet.set_column(0,max_col+1,25)
#             worksheet.set_column(9,9,100)
#             worksheet.set_column(3,3,30)
#             worksheet.set_column(13,13,85)
            print("Done")


2024-01-09 10:16:18.229891-[get_report] delete D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\RSL File to send email\By Brand\Chaco Korea PO Detail-RSL Expiry.xlsx
2024-01-09 10:16:18.231999-[get_report] delete D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\RSL File to send email\By Brand\Harley Korea PO Detail-RSL Expiry.xlsx
2024-01-09 10:16:18.233724-[get_report] delete D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\RSL File to send email\By Brand\Hush Puppies Korea PO Detail-RSL Expiry.xlsx
2024-01-09 10:16:18.235486-[get_report] delete D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\RSL File to send email\By Brand\Keds Korea PO Detail-RSL Expiry.xlsx
2024-01-09 10:16:18.238088-[get_report] delete D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Da

In [23]:
#Split file by Brand+Factory group:
for folder_path in os.listdir(by_factory_path):
        if folder_path.endswith('.xlsx'):
            del_path=f"{by_factory_path}\\{folder_path}"
            modification_time=os.path.getmtime(del_path)
            os.remove(del_path)
#             time.sleep(2)
            print(f"{datetime.datetime.now()}-[get_report] delete {del_path}")
for b in brand:
        for f in f_group:
            df1=data_new[(data_new["Brand"]==b)&(data_new["Factory Group"]==f)]
            (max_row,max_col)=df1.shape
            print(df1.shape)
            if max_row>0:
                df1.to_excel(f"{by_factory_path}\\{b} {f} Korea PO Detail-RSL Expiry.xlsx", index=False)
for file in os.listdir(by_factory_path):
    if file.endswith('.xlsx'):
        filepath=f"{by_factory_path}\\{file}"
        df = pd.read_excel(filepath)
        (max_row,max_col)=df.shape
        print(df.shape)
        with pd.ExcelWriter(filepath, engine='xlsxwriter') as writer:
            df.to_excel(writer, sheet_name="Sheet1",startrow=0,index=False)
            auto_adjust_xlsx_column_width(df, writer, sheet_name="Sheet1",margin=0)
#             column_settings = [{'header': column} for column in df.columns]
            workbook  = writer.book
            worksheet = writer.sheets['Sheet1']
#             worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
            worksheet.set_column(0,max_col+1,25)
#             worksheet.set_column(9,9,100)
#             worksheet.set_column(3,3,30)
#             worksheet.set_column(13,13,85)
            print("Done")


2024-02-16 10:31:27.309163-[get_report] delete D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\Korea RSL File to send email\By Factory Group\Chaco D'CLASE Korea PO Detail-RSL Expiry.xlsx
2024-02-16 10:31:27.311032-[get_report] delete D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\Korea RSL File to send email\By Factory Group\Harley FENGDUN-MIRAMA Korea PO Detail-RSL Expiry.xlsx
2024-02-16 10:31:27.312150-[get_report] delete D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\Korea RSL File to send email\By Factory Group\Harley SCS Korea PO Detail-RSL Expiry.xlsx
2024-02-16 10:31:27.312150-[get_report] delete D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\Korea RSL File to send email\By Factory Group\Harley SPORTS WORLD Korea PO Detail-RSL Expiry.xlsx
2024-02-16 10:31:27.31390

(4, 13)
(12, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(34, 13)
(1, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(1, 13)
(1, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(5, 13)
(63, 13)
(6, 13)
(18, 13)
(28, 13)
(20, 13)
(97, 13)
(13, 13)
(4, 13)
(2, 13)
(52, 13)
(6, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(2, 13)
(0, 13)
(0, 13)
(0, 13)

In [24]:
def send_email_with_outlook(receiver_email, subject, body, attachment_file_path,cc_email):
    outlook = win32.Dispatch('Outlook.Application')
    mail = outlook.CreateItem(0)
    mail.To = receiver_email
    mail.CC = cc_email
    mail.Subject = subject
    mail.htmlBody = body 
    # Đính kèm tệp
    for file in attachment_file_path:
        mail.Attachments.Add(file)
    # Gửi email
    mail.Send()

In [ ]:
if __name__ == "__main__":
    for attachment in os.listdir(by_factory_path):
        if attachment.endswith('.xlsx'):
            file_path=f"{by_factory_path}\\{attachment}"
            df_ctn = pd.read_excel(file_path)
            df_ctn=pd.merge(df_ctn,df,how='left',on=['Brand','Factory Group'])
            df_ctn["All_email"]=df_ctn["All_email"].str.replace("\n",";")
            name_file=attachment.split("Korea")[0]
            receiver_email=df_ctn["All_email"].unique()[0]
            print(df_ctn.shape)
            subject = f'{name_file}Korea PO Detail RSL Expiry - {dt_string}'
            body = f'<h4>Hi All,<br> Please kindly find today {name_file} Korea PO Detail RSL Expiry as attached or view report of today by clicking to "Go to report".</h4>'
            body+='''
                <p class=MsoNormal align=center
                style='text-align:left;line-height:15.0pt'><span
                  style='font-size:10.5pt;font-family:"Segoe UI",sans-serif;color:white'><a
                    href="https://app.powerbi.com/links/VJ3Xr0cBJP?ctid=deace5d6-717b-4f79-ab12-6357206c0c36&pbi_source=linkShare"><span
                      style='font-size:12.0pt;font-family:"Segoe UI Semibold",sans-serif;color:white;border:solid #117865 6.0pt;padding:0in;background:#117865;text-decoration:none'>Go
                      to report &gt;</span></a>
                  <o:p></o:p>
                </span></p>
                '''
            body+='<body>THIS EMAIL IS AUTOMATICALLY GENERATED BY WOLVERINE PRODUCT ANALYTIC SYSTEM.</body>'
            body+='''
                 <p class=MsoNormal><b><span style='font-size:10.0pt;font-family:"Century Gothic",sans-serif;color:#1F3864'>Thanks
                        and best regards</span></b><b>
                      <span style='font-size:10.0pt;font-family:"Century Gothic",sans-serif'>,</span></b><b><span
                        style='font-size:12.0pt;font-family:"Century Gothic",sans-serif'><br><span style='color:#1F3864'>Tam Nguyen
                          &#8211;</span></span></b><b><span style='font-size:9.0pt;font-family:"Century Gothic",sans-serif;color:#A6A6A6'>
                      </span></b><b><span
                        style='font-size:9.0pt;font-family:"Century Gothic",sans-serif;color:#A6A6A6;mso-fareast-language:ZH-CN'>Product
                        Analytics</span></b><b>
                      <span style='font-size:12.0pt;font-family:"Century Gothic",sans-serif'><br></span></b><b><span
                        style='font-size:9.0pt;font-family:"Century Gothic",sans-serif;color:#1F3864'>TBS Development Center<br>5A Xuyên Á
                        Highway, Di An<br> Binh Duong, Vietnam</span></b>
                    <b>
                      <br>
                      <span style='font-size:10pt;font-family:"Century Gothic",sans-serif;color:#1F3864'>P: +84
                      </span>
                    </b><b>
                      <span style='font-size:9.0pt;font-family:"Century Gothic",sans-serif'>347 413
                        204
                      </span></b><b>
                      <span style='font-size:12.0pt;font-family:"Century Gothic",sans-serif'><br></span></b><a
                      href="mailto:tam.nguyen@wwwinc.com"><b>
                        <span
                          style='font-size:9.0pt;font-family:"Century Gothic",sans-serif;mso-fareast-language:ZH-CN'>tam.nguyen@wwwinc.com</span></b></a><b><span
                        style='font-size:12.0pt;font-family:"Century Gothic",sans-serif;color:#1F497D'>
                        <o:p></o:p>
                      </span></b>
                  </p>
          '''
            attachment_file_path=[f"{by_factory_path}\\{attachment}",r"D:\User\nguyenta\Wolverine World Wide\New Asia Product Operations - Data - Power BI - Data - Power BI\Korea RSL File to send email\Korean RS Policy.pdf"]
            carbon_cop_email="tamnguyen1395@*******"
            if str(receiver_email)!='nan':
                print(receiver_email)
                send_email_with_outlook(receiver_email, subject, body, attachment_file_path,cc_email)